In [1]:
import os
import pandas as pd
import numpy as np
import time

In [2]:
current_dir = os.getcwd()

In [3]:
os.chdir(os.path.join(os.getcwd(), "..", "..", "dev", "data"));

In [4]:
team_id_df = pd.read_csv("team_id_map.csv");

In [5]:
player_id_df = pd.read_csv("player_id_map.csv");

In [6]:
os.chdir(current_dir);

In [7]:
seasons = ["2018-19"]

In [8]:
teams_in_season = {
    2016: {
        "teams": ["Arsenal", "Liverpool", "Chelsea", "Bournemouth", "Crystal Palace", "Manchester City", "Watford",
            "Manchester United", "Tottenham Hotspur", "Everton", "West Bromwich Albion", "Southampton",
            "Leicester City", "Sunderland", "Hull City", "Swansea City", "Stoke City",
            "Middlesbrough", "West Ham United", "Burnley"],
        "champion_ls": "Leicester City",
        "top5_ls": ["Leicester City", "Arsenal", "Tottenham Hotspur", "Manchester City", "Manchester United"],
        "bottom5_ls": ["Watford", "West Bromwich Albion", "Crystal Palace", "Bournemouth", "Sunderland"],
        "promoted_ts": ["Middlesbrough", "Hull City", "Burnley"]
    },
    2017: {
        "teams": ["Arsenal", "Liverpool", "Chelsea", "Bournemouth", "Crystal Palace", "Manchester City", "Watford",
            "Manchester United", "Tottenham Hotspur", "Everton", "West Bromwich Albion", "Southampton",
            "Leicester City", "Newcastle United", "Brighton and Hove Albion", "Swansea City", "Stoke City",
            "Huddersfield Town", "West Ham United", "Burnley"],
        "champion_ls": "Chelsea",
        "top5_ls": ["Chelsea", "Tottenham Hotspur", "Manchester City", "Liverpool", "Arsenal"],
        "bottom5_ls": ["Stoke City", "Crystal Palace", "Swansea City", "Burnley", "Watford"],
        "promoted_ts": ["Newcastle United", "Brighton and Hove Albion", "Huddersfield Town"]
    },
    2018: {
        "teams": ["Arsenal", "Liverpool", "Chelsea", "Bournemouth", "Crystal Palace", "Manchester City", "Watford",
            "Manchester United", "Tottenham Hotspur", "Everton", "Wolverhampton Wanderers", "Southampton",
            "Leicester City", "Newcastle United", "Brighton and Hove Albion", "Cardiff City", "Fulham",
            "Huddersfield Town", "West Ham United", "Burnley"],
        "champion_ls": "Manchester City",
        "top5_ls": ["Manchester City", "Manchester United", "Tottenham Hotspur", "Liverpool", "Chelsea"],
        "bottom5_ls": ["West Ham United", "Watford", "Brighton and Hove Albion", "Huddersfield Town", "Southampton"],
        "promoted_ts": ["Wolverhampton Wanderers", "Cardiff City", "Fulham"]
    },
    2019: {
        "teams": ["Arsenal", "Liverpool", "Chelsea", "Bournemouth", "Crystal Palace", "Manchester City", "Watford",
            "Manchester United", "Tottenham Hotspur", "Everton", "Wolverhampton Wanderers", "Southampton",
            "Leicester City", "Newcastle United", "Brighton and Hove Albion", "Norwich City", "Aston Villa",
            "Sheffield United", "West Ham United", "Burnley"],
        "champion_ls": "Manchester City",
        "top5_ls": ["Manchester City", "Liverpool", "Chelsea", "Tottenham Hotspur", "Arsenal"],
        "bottom5_ls": ["Newcastle United", "Bournemouth", "Burnley", "Southampton", "Brighton and Hove Albion"],
        "promoted_ts": ["Norwich City", "Aston Villa", "Sheffield United"]
    }
}

In [9]:
# -1 means L
# 0 means D
# 1 means W
def getResult(row):
    if (row["team_h_score"] == row["team_a_score"]):
        return 0;
    elif (row["at_home"] == 1 and row["team_h_score"] < row["team_a_score"]):
        return -1;
    elif (row["at_home"] == 1 and row["team_h_score"] > row["team_a_score"]):
        return 1;
    elif (row["at_home"] == 0 and row["team_h_score"] < row["team_a_score"]):
        return 1;
    elif (row["at_home"] == 0 and row["team_h_score"] > row["team_a_score"]):
        return -1;

In [10]:
# main columns
cols = ["season", "GW", "at_home", "player_id", "opponent_id", "champion_ls", "top5_ls", "bottom5_ls", "promoted_ts",
        "influence", "creativity", "threat", "ict_index"];

In [11]:
# to remove columns after data augmented
to_remove_cols = ["minutes", "goals_scored", "goals_conceded", "assists", "clean_sheets", "own_goals", "yellow_cards", "red_cards", "penalties_saved", "penalties_missed",
       "saves", "bonus", "bps", "result"];

In [12]:
# reset column orders
reset_cols = ["season", "GW", "at_home", "player_id", "opponent_id", "champion_ls", "top5_ls", "bottom5_ls", 
              "promoted_ts", 'assists', 'assists_pgw', 'bonus', 'bonus_pgw', 'bps',
       'bps_pgw', 'clean_sheets', 'clean_sheets_pgw',
        'creativity', 'creativity_pgw', 'goals_conceded',
       'goals_conceded_pgw', 'goals_scored', 'goals_scored_pgw', 'ict_index',
       'ict_index_pgw', 'influence', 'influence_pgw', 'minutes', 'minutes_pgw',
       'own_goals', 'own_goals_pgw', 'penalties_missed', 'penalties_missed_pgw',
       'penalties_saved', 'penalties_saved_pgw', 'red_cards', 'red_cards_pgw', 'result', 'result_pgw', 'saves',
       'saves_pgw', 'threat',
       'threat_pgw','yellow_cards', 'yellow_cards_pgw', 'total_points_pgw', 'total_points']

In [13]:
# columns to fix - change the type of it to be int
cols_to_fix = ["GW", "at_home", "player_id", "opponent_id", "champion_ls", "top5_ls", "bottom5_ls", 
               "promoted_ts", "total_points"]

In [14]:
for season in seasons:
    print("Preparing data for season: {}".format(season));
    print('*' * 100);
    
    # set the target season as an int version: 2016-17 -> 2016
    target_season = int("".join(season.split("-")[:-1]));
    
    # get the list of teams in that season and sort them
    teams = teams_in_season[target_season]["teams"];
    teams.sort();
    
    # get the champions, top5, bottom5 and promoted teams for that season
    champion_ls = teams_in_season[target_season]["champion_ls"];
    top5_ls = teams_in_season[target_season]["top5_ls"];
    bottom5_ls = teams_in_season[target_season]["bottom5_ls"];
    promoted_ts = teams_in_season[target_season]["promoted_ts"];
    
    # change to the season data directory
    os.chdir(os.path.join(os.getcwd(), "..", "..", "data", season, "gws"));
    
    # read the merged dataframe
    df = pd.read_csv("merged_gw_data.csv");
    
    # change to the data directory in the dev folder
    os.chdir(current_dir);
    os.chdir(os.path.join(os.getcwd(), "..", "..", "dev", "data"));
    
    # set the season
    df["season"] = target_season;
    
    # set team from the list of teams that season
    print(">> Setting opponent team names ...");
    %time df["team"] = df["opponent_team"].apply(lambda team: teams[team-1]);
    print("✓ Done.");
    
    # encode the teams into id
    print(">> Encoding opponent team names into team ids ...");
    %time df["opponent_id"] = df["team"].apply(lambda team: team_id_df[team_id_df["team_name"] == team]["team_id"].iloc[0]);
    print("✓ Done.");
    
    # set column for is champions last season
    print(">> Creating column for champions last season ...");
    %time df["champion_ls"] = df["team"].apply(lambda team: 1 if team == champion_ls else 0);
    print("✓ Done.");
    
    # set column for top 5 last season
    print(">> Creating column for top 5 last season ...");
    %time df["top5_ls"] = df["team"].apply(lambda team: 1 if team in top5_ls else 0);
    print("✓ Done.");
    
    # set column for bottom 5 last season
    print(">> Creating column for bottom 5 last season ...");
    %time df["bottom5_ls"] = df["team"].apply(lambda team: 1 if team in bottom5_ls else 0);
    print("✓ Done.");
    
    # set column for promoted teams this season
    print(">> Creating column for promoted teams this season ...");
    %time df["promoted_ts"] = df["team"].apply(lambda team: 1 if team in promoted_ts else 0);
    print("✓ Done.");
    
    # drop the opponent team and team columns
    print(">> Dropping team and opponent_team columns ...");
    %time df.drop(["team", "opponent_team"], axis=1, inplace=True);
    print("✓ Done.");
    
    # encode the players into id
    print(">> Encoding player names into player ids ...");
    %time df["player_id"] = df["name"].apply(lambda player: player_id_df[player_id_df["player_name"] == player]["player_id"].iloc[0]);
    print("✓ Done.");
    
    # encode was_home into labels
    print(">> Encoding at home into boolean labels ...");
    %time df["at_home"] = df["was_home"].apply(lambda at_home: 0 if not at_home else 1);
    print("✓ Done.");
    
    # encode the result into -1, 0, 1
    print(">> Creating a column for game result ...");
    %time df["result"] = df.apply(lambda row: getResult(row), axis=1);
    print("✓ Done.");
    
    # drop the player name and was home columns
    print(">> Dropping player name and was home columns ...");
    %time df.drop(["name", "was_home"], axis=1, inplace=True);
    print(">> Done.");
    
    # drop the id column
    if 'id' in df.columns:
        print(">> Dropping id columns ...");
        %time df.drop(["id"], axis=1, inplace=True);
        print("✓ Done.");
        
    # set the target column
    target_col = ["total_points"];
    
    # set columns to be all the columns
    cols = cols + to_remove_cols + target_col;
    
    # create a subset dataframe with the new columns
    current_season = df[cols];
    
    # sort the subset dataframe
    current_season.sort_values(["GW"], inplace=True);
    current_season.reset_index(inplace=True, drop=True);
    
    # create target previous gameweek columns
    target_pgw = ["total_points", "ict_index", "influence", "creativity", "threat"];
    target_pgw = target_pgw + to_remove_cols;
    player_ids = np.unique(current_season["player_id"]);
    
    # create a new dataframe that will be augmented to
    new_df = pd.DataFrame(columns=list(current_season.columns));
    
    # loop through each unique player and augment the data
    print(">> Augmenting player data for each gameweek ...");
    print('-' * 50);
    for player_id in player_ids:
        now = time.time();
        df_ = current_season[current_season["player_id"] == player_id]
        df_.reset_index(inplace=True, drop=True)
        for index in range(len(df_)):
            instance = df_.iloc[index]
            gw = instance["GW"];
            for target in target_pgw:
                if (gw == 1 or index == 0):
                    df_.ix[index, "{}_pgw".format(target)] = 0
                else:
                    df_.ix[index, "{}_pgw".format(target)] = df_.iloc[index-1][target]
        new_df = new_df.append(df_)
        later = time.time();
        difference = int(later - now);
        print("Completed player id {} --- Time taken: {}s".format(player_id, difference));
    print("✓ Done.");
    print('-' * 50);
    
    # fix the indices
    new_df.reset_index(inplace=True, drop=True);
    
    # subset the reset columns
    new_df = new_df[reset_cols];
    
    # fix the reset columns
    for col in cols_to_fix:
        new_df[col] = new_df[col].astype(int)
    
    # save the dataframe
    filename = "{}-prepared-data.csv".format(target_season);
    new_df.to_csv(filename, index=False);
    
    print("Done preparing data for {}. File saved at: {}".format(season, os.path.join(os.getcwd(), filename)));
    print('*' * 100);
    print("\n");
    
    # switch back to the scripts directory
    os.chdir(current_dir);

Preparing data for season: 2018-19
****************************************************************************************************
>> Setting opponent team names ...
CPU times: user 3.93 ms, sys: 87 µs, total: 4.01 ms
Wall time: 3.97 ms
✓ Done.
>> Encoding opponent team names into team ids ...
CPU times: user 12.2 s, sys: 178 ms, total: 12.4 s
Wall time: 12.3 s
✓ Done.
>> Creating column for champions last season ...
CPU times: user 6.88 ms, sys: 106 µs, total: 6.98 ms
Wall time: 6.93 ms
✓ Done.
>> Creating column for top 5 last season ...
CPU times: user 8.09 ms, sys: 117 µs, total: 8.2 ms
Wall time: 8.11 ms
✓ Done.
>> Creating column for bottom 5 last season ...
CPU times: user 8.37 ms, sys: 205 µs, total: 8.57 ms
Wall time: 8.48 ms
✓ Done.
>> Creating column for promoted teams this season ...
CPU times: user 7.57 ms, sys: 148 µs, total: 7.72 ms
Wall time: 7.63 ms
✓ Done.
>> Dropping team and opponent_team columns ...
CPU times: user 9.43 ms, sys: 5.95 ms, total: 15.4 ms
Wall ti

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:125: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/site-packages/pandas/

Completed player id 1 --- Time taken: 17s
Completed player id 2 --- Time taken: 16s
Completed player id 4 --- Time taken: 16s
Completed player id 5 --- Time taken: 16s
Completed player id 6 --- Time taken: 16s
Completed player id 12 --- Time taken: 18s
Completed player id 15 --- Time taken: 16s
Completed player id 17 --- Time taken: 16s
Completed player id 18 --- Time taken: 16s
Completed player id 21 --- Time taken: 15s
Completed player id 22 --- Time taken: 16s
Completed player id 26 --- Time taken: 16s
Completed player id 28 --- Time taken: 16s
Completed player id 33 --- Time taken: 16s
Completed player id 34 --- Time taken: 16s
Completed player id 35 --- Time taken: 16s
Completed player id 38 --- Time taken: 16s
Completed player id 40 --- Time taken: 18s
Completed player id 41 --- Time taken: 16s
Completed player id 46 --- Time taken: 18s
Completed player id 47 --- Time taken: 17s
Completed player id 49 --- Time taken: 16s
Completed player id 51 --- Time taken: 16s
Completed player

Completed player id 455 --- Time taken: 15s
Completed player id 456 --- Time taken: 16s
Completed player id 463 --- Time taken: 17s
Completed player id 464 --- Time taken: 16s
Completed player id 466 --- Time taken: 17s
Completed player id 472 --- Time taken: 16s
Completed player id 474 --- Time taken: 16s
Completed player id 478 --- Time taken: 16s
Completed player id 483 --- Time taken: 17s
Completed player id 484 --- Time taken: 15s
Completed player id 485 --- Time taken: 6s
Completed player id 486 --- Time taken: 17s
Completed player id 490 --- Time taken: 16s
Completed player id 491 --- Time taken: 17s
Completed player id 493 --- Time taken: 17s
Completed player id 497 --- Time taken: 15s
Completed player id 498 --- Time taken: 16s
Completed player id 500 --- Time taken: 16s
Completed player id 503 --- Time taken: 14s
Completed player id 504 --- Time taken: 14s
Completed player id 505 --- Time taken: 15s
Completed player id 507 --- Time taken: 2s
Completed player id 508 --- Time t

Completed player id 801 --- Time taken: 15s
Completed player id 804 --- Time taken: 11s
Completed player id 805 --- Time taken: 15s
Completed player id 806 --- Time taken: 15s
Completed player id 808 --- Time taken: 6s
Completed player id 809 --- Time taken: 14s
Completed player id 814 --- Time taken: 5s
Completed player id 815 --- Time taken: 14s
Completed player id 817 --- Time taken: 15s
Completed player id 820 --- Time taken: 15s
Completed player id 822 --- Time taken: 15s
Completed player id 825 --- Time taken: 15s
Completed player id 827 --- Time taken: 15s
Completed player id 829 --- Time taken: 15s
Completed player id 830 --- Time taken: 16s
Completed player id 832 --- Time taken: 14s
Completed player id 833 --- Time taken: 14s
Completed player id 834 --- Time taken: 0s
Completed player id 835 --- Time taken: 14s
Completed player id 836 --- Time taken: 15s
Completed player id 839 --- Time taken: 10s
Completed player id 840 --- Time taken: 15s
Completed player id 841 --- Time ta

Completed player id 1032 --- Time taken: 8s
Completed player id 1033 --- Time taken: 16s
Completed player id 1034 --- Time taken: 8s
Completed player id 1035 --- Time taken: 16s
Completed player id 1036 --- Time taken: 6s
Completed player id 1037 --- Time taken: 16s
Completed player id 1038 --- Time taken: 16s
Completed player id 1039 --- Time taken: 16s
Completed player id 1040 --- Time taken: 17s
Completed player id 1041 --- Time taken: 17s
Completed player id 1042 --- Time taken: 17s
Completed player id 1043 --- Time taken: 0s
Completed player id 1044 --- Time taken: 2s
Completed player id 1045 --- Time taken: 14s
Completed player id 1046 --- Time taken: 17s
Completed player id 1047 --- Time taken: 5s
Completed player id 1048 --- Time taken: 17s
Completed player id 1049 --- Time taken: 16s
Completed player id 1050 --- Time taken: 16s
Completed player id 1051 --- Time taken: 17s
Completed player id 1052 --- Time taken: 7s
Completed player id 1053 --- Time taken: 16s
Completed player 